In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import HistGradientBoostingRegressor
import matplotlib.pyplot as plt

In [43]:
# Load and preview the dataset
# Update the file path to match your dataset location
file_path = './DBtrainrides_final_result.csv'
train_ride_df = pd.read_csv(file_path)
train_ride_df.head()

,ID_Base,ID_Timestamp,stop_number,IBNR,long,lat,arrival_plan,departure_plan,arrival_delay_m,transformed_info_message,prev_arrival_delay_m,prev_departure_delay_m,weighted_avg_prev_delay,max_station_number,station_progress
0,-1001326572688500578,2407082041,2,8011118.0,13.375988,52.509379,2024-07-08 20:44:00,2024-07-08 20:45:00,0.0,No message,0.0,0.0,0.000000,7,0.285714
1,-1001326572688500578,2407082041,3,8011160.0,9.095851,48.849792,NaN,NaN,NaN,No message,0.0,0.0,0.000000,7,0.428571
2,-1001326572688500578,2407082041,4,8011167.0,13.299437,52.530276,2024-07-08 20:55:00,2024-07-08 20:56:00,0.0,No message,0.0,0.0,0.000000,7,0.571429
3,-1001326572688500578,2407082041,5,8010404.0,13.196898,52.534648,2024-07-08 21:00:00,2024-07-08 21:03:00,2.0,No message,0.0,0.0,0.000000,7,0.714286
4,-1001326572688500578,2407082041,6,8080040.0,13.128917,52.549396,2024-07-08 21:06:00,2024-07-08 21:07:00,1.0,No message,2.0,0.0,0.666667,7,0.857143


In [44]:
# Handle missing values and prepare features and target
# Define features and target variable
features = [
    'stop_number', 'IBNR', 'long', 'lat', 
    'prev_arrival_delay_m', 'prev_departure_delay_m', 
    'weighted_avg_prev_delay', 'max_station_number', 'station_progress'
]
target = 'arrival_delay_m'

X = train_ride_df[features]
y = train_ride_df[target]

# Handle missing values in the target variable
if y.isnull().any():
    print(f"Number of missing values in target: {y.isnull().sum()}")
    # Option 1: Impute missing values in y
    y.fillna(y.mean(), inplace=True)
    # Or use Option 2: Drop rows with missing target values
    # train_ride_df.dropna(subset=[target], inplace=True)
    # X = train_ride_df[features]
    # y = train_ride_df[target]

Number of missing values in target: 831630


In [45]:
# Display the feature matrix and target to verify
X.head()

,stop_number,IBNR,long,lat,prev_arrival_delay_m,prev_departure_delay_m,weighted_avg_prev_delay,max_station_number,station_progress
0,2,8011118.0,13.375988,52.509379,0.0,0.0,0.000000,7,0.285714
1,3,8011160.0,9.095851,48.849792,0.0,0.0,0.000000,7,0.428571
2,4,8011167.0,13.299437,52.530276,0.0,0.0,0.000000,7,0.571429
3,5,8010404.0,13.196898,52.534648,0.0,0.0,0.000000,7,0.714286
4,6,8080040.0,13.128917,52.549396,2.0,0.0,0.666667,7,0.857143


In [46]:
y.head()

0    0.000000
1    1.255314
2    0.000000
3    2.000000
4    1.000000
Name: arrival_delay_m, dtype: float64

In [47]:
# Check for missing values
print("Missing values in features:\n", X.isnull().sum())
print("Missing values in target:", y.isnull().sum())

Missing values in features:
 stop_number                     0
IBNR                       121088
long                            0
lat                             0
prev_arrival_delay_m            0
prev_departure_delay_m          0
weighted_avg_prev_delay         0
max_station_number              0
station_progress                0
dtype: int64
Missing values in target: 0


In [48]:
# Hyperparameter tuning using k-fold cross-validation
# Define the HistGradientBoostingRegressor model
hgbr = HistGradientBoostingRegressor()

# Define hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [10, 20, 30]
}

# Use GridSearchCV for hyperparameter tuning
scorer = make_scorer(r2_score)
grid_search = GridSearchCV(estimator=hgbr, param_grid=param_grid, scoring=scorer, cv=5, verbose=1)
grid_search.fit(X, y)

# Display the best hyperparameters and the corresponding score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Cross-Validation R-squared Score: {best_score}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'max_iter': 300, 'min_samples_leaf': 10}
Best Cross-Validation R-squared Score: 0.6780548160289204


In [49]:
# Evaluate the best model using k-fold cross-validation
# Use the best estimator from GridSearchCV
best_hgbr = grid_search.best_estimator_

# Perform k-fold cross-validation
cv_scores = cross_val_score(best_hgbr, X, y, cv=5, scoring='r2')

# Print cross-validation results
print(f"Cross-Validation R-squared Scores: {cv_scores}")
print(f"Mean R-squared Score: {np.mean(cv_scores)}")
print(f"Standard Deviation of R-squared Scores: {np.std(cv_scores)}")

Cross-Validation R-squared Scores: [0.68507323 0.6750879  0.67859023 0.67770233 0.67771683]
Mean R-squared Score: 0.6788341035494211
Standard Deviation of R-squared Scores: 0.0033331517953366424


In [50]:
# Feature importance analysis
# Extract and visualise feature importance
feature_importances = best_hgbr.feature_importances_
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Display the feature importances
importance_df

AttributeError: 'HistGradientBoostingRegressor' object has no attribute 'feature_importances_'

In [ ]:
# Plot feature importances
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance in HistGradientBoostingRegressor')
plt.gca().invert_yaxis()  # Reverse the order for better readability
plt.show()

In [ ]:
# Residual analysis
# Predict on the full dataset to check residual patterns
y_pred = best_hgbr.predict(X)
residuals = y - y_pred

# Scatter plot of residuals
plt.scatter(y, residuals, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('True Values')
plt.ylabel('Residuals')
plt.title('Residual Analysis')
plt.show()